In [37]:
%rm -r sample_data
!wget --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
!mkdir -p ffhq/train/real
!mkdir -p ffhq/train/synthesized
!mkdir -p ffhq/test/real
!mkdir -p ffhq/test/synthesized
%cd /content/0_real/
!mv $(ls | shuf -n 2500) ../ffhq/train/real/
!mv * ../ffhq/test/real/
%cd ..
!rm -d 0_real

/content/0_real
/content


In [ ]:
!%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf

/bin/bash: line 0: fg: no job control


In [ ]:
batch_size = 32
interpolation = 'bilinear'
train_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=1/5, # Validation is 1/6 of the total, so 1/5 of (total - test)
    subset='training',
    interpolation=interpolation,
    crop_to_aspect_ratio=True
)
val_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=1/5,
    subset='validation',
    interpolation=interpolation,
    crop_to_aspect_ratio=True
)

In [ ]:
# Preprocessing
!pip install tensorflow-addons
import tensorflow_addons as tfa
def randomGaussianBlur(x):
    k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.dtypes.float32
    )
    return tfa.image.gaussian_filter2d(
        x,
        filter_shape = (3, 3),
        sigma = k,
        padding = 'REFLECT'
    )

def randomRot90(x):
    k = tf.random.uniform(
        shape=[],
        minval=0,
        maxval=4,
        dtype=tf.dtypes.uint8,
        seed=None,
        name=None
    )
    return tf.image.rot90(x, k)

def preprocess(x):
    y = tf.keras.layers.CenterCrop(224,224)(x)
    y = tf.keras.layers.Rescaling(1./127.5, offset=-1)(x)
def augment(x):
    y = randomGaussianBlur(x)
    y = randomRot90(x)

train_dataset.map(lambda x, y: (preprocess(x), y))
val_dataset.map(lambda x, y: (preprocess(x), y))
train_dataset.map(lambda x, y: (augment(x), y))

In [ ]:
# Reference: https://www.tensorflow.org/tutorials/load_data/images#configure_the_dataset_for_performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.applications.Xception(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None, # hyperparameter?
)
train_dataset = tf.keras.applications.xception.preprocess_input(train_dataset)
val_dataset = tf.keras.applications.xception.preprocess_input(val_dataset)

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
)
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction=losses_utils.ReductionV2.AUTO,
    name='binary_crossentropy'
)
metric = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='ROC',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False
)
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metric
)

In [ ]:
batch_size=32
epochs = 1
model.fit(
    train_dataset,
    batch_size=batch_size,
    epochs=epochs,
    verbose="auto",
    validation_data=val_dataset,
    shuffle=True,
    validation_freq=1
)